In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as col

import ksource as ks
import mcpl

# Create KSource

### Input cell:
It is recommiended to complete template in a duplicated cell

In [ ]:
# Open particle list

plfile = "plistfilename"
readformat = "mcpl" # "mcpl", "ssw", "phits", "ptrac", "stock"
trasl = None
rot = None
x2z = False
plist = ks.PList(plfile, readformat, trasl=trasl, rot=rot, switch_x2z=x2z)

# Create Geometry

trasl = None
rot = None
geom = ks.GeomFlat(trasl=trasl, rot=rot)

# Create KSource

J = 1 # Source intensity [1/s]
s = ks.KSource(plist, geom, J=J)

# Evaluate statistics

In [ ]:
# With MCPL

file = s.plist.filename
mcpl.dump_stats(mcpl.collect_stats(file))
mcpl.plot_stats(file)

In [ ]:
# With ks.Stats

# Select variables ranges (box)
dxymax = -1
dzmin = np.sqrt(1-dxymax**2)
maskfun = ks.Box([None,None,None,None,None,None,dzmin], None)

# Weighting function
fact_dosim = ks.H10(pt=plist.pt)
weightfun = lambda parts: fact_dosim(parts[:,0])

parts,ws = plist.get()
stats = ks.Stats(parts, ws, weightfun=weightfun, maskfun=maskfun)

N,I,err = stats.mean_weight(steps=100)
plt.show()
N,mn,err = stats.mean(var=1, steps=100)
plt.show()
N,std,err = stats.std(var=1, steps=100)
plt.show()

# Optimize BW

In [ ]:
# Set importance for each variable

var_imp = [1]*geom.dim # Default: all importances = 1
parts,ws = s.plist.get(1E4)
scaling = s.geom.std(parts=parts, weights=ws)
scaling /= var_imp

In [ ]:
# Number of particles to use for optimization.
# Use -1 to use all particles in list (recommended)
N = -1

Choose one of the available bandwidth optimization methods. Recommended method is Method 3 (adaptive MLCV)

In [ ]:
# Method 1: Silverman's Rule: Simple and fast method.
# BW is chosen based on only on the number of particles, and dimension of
# geometry.

s.bw_method = "silv"
s.fit(N, scaling=scaling)

In [ ]:
# Method 2: Non-adaptive Maximum Likelihood Cross-Validation:
# Creates a grid of non-adaptive bandwidths and evaluates the
# cross-validation scores on each one, which is an indicator of the
# quality of the estimation. Selects the bandwidth that optimizes
# CV score.

# MLCV optimization with reduced N
s.bw_method = "mlcv"
grid = np.logspace(-1,1,20)
N_cv = int(1E4) # Use a smaller N to reduce computation times
s.fit(N_cv, scaling=scaling, grid=grid)

# Adapt optimized bandwidth
bw = s.kde.bw
dim = s.geom.dim
bw *= ks.bw_silv(dim,N)/ks.bw_silv(dim,N_cv) # Apply Silverman factor
s = ks.KSource(plist, geom, bw=bw) # Create new KSource with adapted BW
s.fit(N=N, scaling=scaling)

In [ ]:
# Method 3: Adaptive Maximum Likelihood Cross-Validation:
# Creates a grid of adaptive bandwidths and evaluates the
# cross-validation scores on each one, which is an indicator of the
# quality of the estimation. Selects the bandwidth that optimizes
# CV score.
# kNN is used to generate the seed adaptive bandwidth.

# kNN bandwidth
s.bw_method = "knn"
batch_size = 10000 # Batch size for KNN search
k = 10             # Numer of neighbors per batch
s.fit(N, scaling=scaling, batch_size=batch_size, k=k)
bw_knn = s.kde.bw

# MLCV optimization of previously calculated kNN bandwidth
s.bw_method = "mlcv"
N_cv = int(1E4)      # Use a smaller N to reduce computation times
seed = bw_knn[:N_cv] # Use kNN BW as seed (first N elements)
grid = np.logspace(-1,1,20)
s.fit(N_cv, scaling=scaling, seed=seed, grid=grid)
bw_cv = s.kde.bw

# Extend MLCV optimization to full KNN BW
bw_knn_cv = bw_knn * bw_cv[0]/bw_knn[0]   # Apply MLCV factor
dim = s.geom.dim
bw_knn_cv *= ks.bw_silv(dim,len(bw_knn))/ks.bw_silv(dim,len(bw_cv)) # Apply Silverman factor
s = ks.KSource(plist, geom, bw=bw_knn_cv) # Create new KSource with full BW
s.fit(N=N, scaling=scaling)

# Save KSource

In [ ]:
xmlfilename = None # XML source parameters file name
bwfilename = None  # BW file name. Only for adaptive BW

xmlfilename = s.save(xmlfilename, bwfilename)

# Compare tracks and resampled

### Sample particles from KDE source

In [ ]:
resampledfile = tracksfile.split('.')[0]+"_resampled"
N_resampled = N
!kstool resample $sourcefilename -n $N_resampled -o $resampledfile
resampledfile += ".mcpl.gz"

### Generate histograms of the tracks list and resampled list
Modify templates to build the desired plots to compare the tracks and KDE distributions

#### Fast 1D histograms using mcpl library

In [ ]:
var = "x"

# Tracks list histogram
bins_tracks,hist = stats_tracks[var]["hist_bins"],stats_tracks[var]["hist"]
hist_tracks /= np.sum(hist_tracks)
plt.stairs(hist, edges=bins_tracks, baseline=None, label="Original")

# Resampled list histogram
bins_kde,hist = stats_resampled[var]["hist_bins"],stats_resampled[var]["hist"]
hist_kde /= np.sum(hist_kde)
plt.stairs(hist, edges=bins_kde, baseline=None, label="Resampled")

plt.grid()
plt.legend()
plt.xlabel(var)
plt.ylabel("J")
plt.show()

#### 1D histograms

In [ ]:
nbins = 100
bins = np.logspace(-10,10,nbins+1)
grid = (bins[1:] + bins[:-1]) / 2

# Tracks list histogram
mcplfile = mcpl.MCPLFile(plfile) # plfile should be an MCPL file for this to work
hist_tracks = np.zeros(nbins)
errs_tracks = np.zeros(nbins)
I = 0
for pb in mcplfile.particle_blocks:
    data = pb.ekin
    ws = pb.weight
    wssum = ws.sum()
    I += wssum
    if wssum > 0:
        hist_tracks += np.histogram(data, bins=bins, weights=ws, density=True)[0] * wssum
        errs_tracks += np.histogram(data, bins=bins, weights=ws**2)[0]
hist_tracks *= J / I
errs_tracks = np.sqrt(errs_tracks) / (bins[1:]-bins[:-1]) * J / I * grid
sp=plt.stairs(hist_tracks, edges=bins, baseline=None, label="Tracks")
c=sp.get_edgecolor()
plt.errorbar(grid, hist_tracks, yerr=errs_tracks, capsize=1, linewidth=1, fmt='none', ecolor=c)

# Resampled list histogram
mcplfile = mcpl.MCPLFile(resampledfile)
hist_kde = np.zeros(nbins)
errs_kde = np.zeros(nbins)
I = 0
for pb in mcplfile.particle_blocks:
    data = pb.ekin
    ws = pb.weight
    wssum = ws.sum()
    I += wssum
    if wssum > 0:
        hist_kde += np.histogram(data, bins=bins, weights=ws, density=True)[0] * wssum
        errs_kde += np.histogram(data, bins=bins, weights=ws**2)[0]
hist_kde *= J / I
errs_kde = np.sqrt(errs_kde) / (bins[1:]-bins[:-1]) * J / I * grid
sp=plt.stairs(hist_kde, edges=bins, baseline=None, label="Resampled")
c=sp.get_edgecolor()
plt.errorbar(grid, hist_kde, yerr=errs_kde, capsize=1, linewidth=1, fmt='none', ecolor=c)

plt.grid()
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.xlabel(r"Energy [MeV]")
plt.ylabel(r"$J\ \left[\frac{n}{MeV\ s}\right]$")
plt.show()

#### 2D histograms

In [ ]:
nbins1 = 100
bins1 = np.linspace(-10,10,nbins1+1)
nbins2 = 100
bins2 = np.linspace(-10,10,nbins2+1)

# Tracks list histogram
mcplfile = mcpl.MCPLFile(plfile) # plfile should be an MCPL file for this to work
hist_tracks = np.zeros((nbins2,nbins1))
I = 0
for pb in mcplfile.particle_blocks:
    data1 = pb.x
    data2 = pb.y
    ws = pb.weight
    wssum = ws.sum()
    I += wssum
    if wssum > 0:
        h,_,_ = np.histogram2d(data2, data1, bins=[bins2,bins1], weights=ws, density=True)
        hist_tracks += h * wssum
hist_tracks *= J / I
plt.pcolormesh(bins1, bins2, hist_tracks, cmap='jet', rasterized=True)
plt.title("Tracks")
plt.xlabel("x [cm]")
plt.ylabel("y [cm]")
cbar = plt.colorbar(label=r"$\frac{n}{cm^2 s}$")
plt.show()

# Resampled list histogram
mcplfile = mcpl.MCPLFile(resampledfile)
hist_kde = np.zeros((nbins2,nbins1))
I = 0
for pb in mcplfile.particle_blocks:
    data1 = pb.x
    data2 = pb.y
    ws = pb.weight
    I += ws.sum()
    h,_,_ = np.histogram2d(data2, data1, bins=[bins2,bins1], weights=ws, density=True)
    hist_kde += h * ws.sum()
hist_kde *= J / I
plt.pcolormesh(bins1, bins2, hist_kde, cmap='jet', rasterized=True)
plt.title("KDE")
plt.xscale("log")
plt.xlabel("x [cm]")
plt.ylabel("y [cm]")
cbar = plt.colorbar(label=r"$\frac{n}{cm^2 s}$")
plt.show()

# Create KDE plots
Generate plots of the estimated distribution using the KSource Python API

In [ ]:
# Energy plots

vec0 = None
vec1 = None
EE = np.logspace(-9,0,50)
fig,scores = s.plot_E(EE, vec0, vec1)
plt.show()

In [ ]:
# XY plots

vec0 = None
vec1 = None
xx = np.linspace(-30,30,30)
yy = np.linspace(-30,30,30)
fig,[scores,errs] = s.plot2D_integr(["x","y"], [xx,yy], vec0, vec1)
plt.show()

In [ ]:
# ZT plots (guide)

Lz = s.geom.ms[1].zmax
Lt = 2 * (s.geom.ms[1].xwidth + s.geom.ms[1].yheight)

vec0 = None
vec1 = None 
zz = np.linspace(0,Lz,50)
fig,[scores,errs] = s.plot_integr("z", zz, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
tt = np.linspace(0,Lt,50)
fig,[scores,errs] = s.plot_integr("t", tt, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
zz = np.linspace(0,Lz,30)
tt = np.linspace(0,Lt,30)
fig,[scores,errs] = s.plot2D_integr(["z","t"], [zz,tt], vec0, vec1, scale="log")
plt.show()

In [ ]:
# Polar plots

vec0 = None
vec1 = None
tth = np.linspace(0,90,50)
fig,[scores,errs] = s.plot_integr("theta", tth, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
pp = np.linspace(-180,180,50)
fig,[scores,errs] = s.plot_integr("phi", pp, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
tth = np.linspace(0,90,30)
pp = np.linspace(-180,180,30)
fig,[scores,errs] = s.plot2D_integr(["mu","phi"], [tth,pp], vec0, vec1)
plt.show()

In [ ]:
# Isotrop plots

vec0 = None
vec1 = None
ddz = np.linspace(0.9,1,50)
fig,[scores,errs] = s.plot_integr("dz", ddz, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
ddy = np.linspace(-0.1,0.1,50)
fig,[scores,errs] = s.plot_integr("dy", ddy, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
ddx = np.linspace(-0.1,0.1,20)
ddy = np.linspace(-0.1,0.1,20)
fig,[scores,errs] = s.plot2D_integr(["dx","dy"], [ddx,ddy], vec0, vec1)
plt.show()